## Visualize the node and relationship using Qdrant Vector DB

In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

EMBEDDING_MODEL_NAME = "intfloat/multilingual-e5-large-instruct"
embed_model = HuggingFaceEmbedding(model_name=EMBEDDING_MODEL_NAME, trust_remote_code=True)

d:\Arun N\Personal\understanding-ai-agents\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
text_data = ["Apple", "Orange", "Black", "Realme", "Red", "Yellow", "Soliton Technologies", "Business"]

text_data

['Apple',
 'Orange',
 'Black',
 'Realme',
 'Red',
 'Yellow',
 'Soliton Technologies',
 'Business']

In [4]:
from llama_index.core import Document

chunks = []
for text in text_data:
    doc = Document(text=text)
    doc.embedding = embed_model.get_text_embedding(text)
    chunks.append(doc)

print(f"Number of chunks created: {len(chunks)}")

Number of chunks created: 8


In [5]:
# Use Qdrant-DB from as a vector database
from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "Nodes_AND_Relationships"

def get_vector_store():
    client = qdrant_client.QdrantClient(
        url=QDRANT_URL,

    )

    vector_store = QdrantVectorStore(
        client=client,
        collection_name=COLLECTION_NAME,
    )
    return vector_store

qdrant_vector_store = get_vector_store()

In [6]:
# Add the chunks to the Qdrant vector store

_ = qdrant_vector_store.add(
    nodes=chunks,
    show_progress=True,
)

#### Change the Query Below and test with different queries

Example Queries
- Newton
- Software
- Samsung
- India
- Blue

In [19]:
from llama_index.core import QueryBundle

query = "India"

query_embeddings = embed_model.get_text_embedding(query)

query_bundle = QueryBundle(query_str="", embedding=query_embeddings)

print(len(query_embeddings))

1024


In [20]:
from llama_index.core import VectorStoreIndex

SIMILARITY_TOP_K = 3

vector_store_index = VectorStoreIndex.from_vector_store(
    vector_store=qdrant_vector_store, embed_model=embed_model
)

qdrant_retriever = vector_store_index.as_retriever(similarity_top_k=SIMILARITY_TOP_K)
retrieved_nodes = qdrant_retriever.retrieve(query_bundle)

In [21]:
for node in retrieved_nodes:
    print(f"Retrieved Node: {node.text}\n")
    print(f"Similarity Score: {node.score}\n")
    print("-" * 80)

Retrieved Node: Realme

Similarity Score: 0.848204

--------------------------------------------------------------------------------
Retrieved Node: Red

Similarity Score: 0.8455509

--------------------------------------------------------------------------------
Retrieved Node: Black

Similarity Score: 0.840017

--------------------------------------------------------------------------------
